In [42]:
import pandas as pd
import sqlite3
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np

pd.set_option('display.max_rows', None)

db_path = 'deals_db.db'

In [46]:
def clean_text(text):
    cleaned_text = re.sub(r'\b[a-zA-Z]\b|\b\d+\b', '', text)
    return re.sub(r'\s+', ' ', cleaned_text).strip().lower()

In [48]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""SELECT c.title || " " ||  c.title_general as document
    FROM deals c
""")
rows = cursor.fetchall()

conn.close()

In [52]:
corpus = []
for row in rows:
    if row[0]:
        line = clean_text(row[0])
        corpus.append(line)

In [53]:
corpus[:2]

['sunset tour by kayak on sebago lake maine sunset tour by kayak on sebago lake maine',
 'quarter rotisserie chicken meal savor the flavor: quarter rotisserie chicken meal at texas rotisserie and grill- broadway(up to % off)']

In [55]:
tokens = []
for line in corpus:
    data = word_tokenize(line)
    for word in data:
    #filtered_data = [[word for word in sentence if len(word) > 2] for sentence in data]
        if len(word) > 2:
            tokens.append(word)

In [ ]:
#documents = [' '.join(sentence) for sentence in tokens]
documents = [' '.join(tokens)]

# Inicializace TfidfVectorizer
#tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

# Fit and transform data
#tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Get feature names
#trigrams = tfidf_vectorizer.get_feature_names_out()

# Compute the mean TF-IDF scores
#mean_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).flatten()

#filtered_mean_tfidf = [(trigram, tfidf) for trigram, tfidf in zip(trigrams, mean_tfidf) if tfidf > 0.000001]


# Save to CSV
#output_file = 'models/mean_tfidf_unigram.csv'
#with open(output_file, 'w', encoding='utf-8') as f:
#    f.write('unigram,tfidf\n')
#    for trigram, tfidf in filtered_mean_tfidf:
#       f.write(f'{trigram},{tfidf}\n')
documents[:10]

In [8]:
df = pd.read_csv("models/mean_tfidf_unigram.csv").sort_values(["tfidf"], ascending = False)
df.count()

unigram    51009
tfidf      51009
dtype: int64

In [89]:
filtered_mean_tfidf[:10]

[('002oz57ml', 1.5954199494191315e-05),
 ('003oz11g', 1.5954199494191315e-05),
 ('004oz11g', 4.786259848257395e-05),
 ('007oz22g', 1.5954199494191315e-05),
 ('008oz24g', 1.5954199494191315e-05),
 ('010oz3g', 1.5954199494191315e-05),
 ('012oz35g', 1.5954199494191315e-05),
 ('015ct', 2.3931299241286974e-05),
 ('015mm', 2.3931299241286974e-05),
 ('016oz52ml', 1.5954199494191315e-05)]

In [ ]:
df = pd.read_csv("models/mean_tfidf_unigram.csv").sort_values(["tfidf"], ascending = False)
df.count()
dictionary = df["unigram"].tolist()
dictionary

In [107]:
from fuzzywuzzy import process

dictionary = df["unigram"].tolist()
for w in ["valvoln", "mesage"]:
    corrected_word = process.extractBests(w, dictionary)
    print(corrected_word[0]) # 

('vol', 90)
('message', 92)


In [116]:
df = pd.read_csv("models/dict_uniq.csv")
df.count()
word_list = df["query"].tolist()
word_list[:20]

['amc',
 'atv',
 'atlantic city',
 'bowlero',
 'brazilian',
 'brazilian wax',
 'cancun',
 'cinema',
 'christmas',
 'christmas cards',
 'chuck e. cheese',
 'coolsculpting',
 "save and buster's",
 'disney',
 'disneyland',
 'emsculpt',
 'halloween',
 'ipl',
 'jpcenney',
 'juvederm']

In [119]:
def calculate_similarity(word1, word2):

    counter1 = Counter(word1)
    counter2 = Counter(word2)

    common_letters = set(counter1.keys()) & set(counter2.keys())
    similarity_score = sum(min(counter1[letter], counter2[letter]) for letter in common_letters)

    length_factor = min(len(word1), len(word2)) / max(len(word1), len(word2))

    initial_match_bonus = 0
    for i in range(min(len(word1), len(word2))):
        if word1[i] == word2[i]:
            initial_match_bonus += 1
        else:
            break

    total_similarity = (similarity_score * length_factor) + initial_match_bonus
    return total_similarity


# Test
input_words = ['acm', 'oli', 'oli chance', 'ifone', 'cuple', 'message', 'valentyn', 'pakr' ]

for input_word in input_words:

    similarities = {word: calculate_similarity(input_word, word) for word in word_list}
    most_similar_word = max(similarities, key=similarities.get)

    print(f'Most similar wird to "{input_word}" is "{most_similar_word}" with score {similarities[most_similar_word]}.')


Most similar wird to "acm" is "amc" with score 4.0.
Most similar wird to "oli" is "oil" with score 4.0.
Most similar wird to "oli chance" is "change oil" with score 9.0.
Most similar wird to "ifone" is "iphone" with score 4.333333333333334.
Most similar wird to "cuple" is "couple" with score 5.166666666666667.
Most similar wird to "message" is "massage" with score 7.0.
Most similar wird to "valentyn" is "valentine's day" with score 10.266666666666666.
Most similar wird to "pakr" is "parking" with score 4.285714285714286.
